In [192]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators
import io, base64, parmed, mdtraj
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [68]:
# Generate SMIRNOFF ligand mol object
mol = oechem.OEMol()
molpdb = '../OpenMMCubes/tests/input/toluene.pdb'
ffxml = open('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml', 'rb')
with oechem.oemolistream(molpdb) as ifs:
    oechem.OEReadMolecule(ifs, mol)
    with oechem.oemolostream('smirff_mol.oeb.gz') as ofs:
        mol.SetData(oechem.OEGetTag('forcefield'), ffxml.read())
        oechem.OEWriteMolecule(ofs, mol)

In [69]:
# Read the SMIRNOFF ligand Structure object
from smarty.forcefield import ForceField
with oechem.oemolistream('smirff_mol.oeb.gz') as ifs:
    ffxml = mol.GetData(oechem.OEGetTag('forcefield')).encode()
    with open('mol_parameters.ffxml', 'wb') as out:
        out.write(ffxml)
mol_ff = ForceField(open('mol_parameters.ffxml'))
mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys, xyz=mol_pos)
molecule_structure.residues[0].name = "LIG"
print(molecule_structure)
print(molecule_structure.topology)
print(molecule_structure.residues)

<Structure 15 atoms; 1 residues; 15 bonds; parametrized>
<Topology; 1 chains, 1 residues, 15 atoms, 15 bonds>
ResidueList([
	<Residue LIG[0]; chain=1>
])


In [70]:
#Generate protein Stucture object
proteinpdb = os.path.join('..','OpenMMCubes', 'tests', 'input', 'T4-protein.pdb')
protein = app.PDBFile(proteinpdb)
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')
protein_system = forcefield.createSystem( protein.topology )
protein_structure = parmed.openmm.load_topology( protein.topology, protein_system, xyz=protein.positions)

print(protein_structure)
print(protein_structure.topology)
#print(protein_structure.positions)

<Structure 2634 atoms; 164 residues; 2654 bonds; parametrized>
<Topology; 1 chains, 164 residues, 2634 atoms, 2654 bonds>


In [71]:
# Merge structures
pl_structure = protein_structure + molecule_structure
print(len(pl_structure.positions))

2649


In [72]:
# Concatenate positions arrays
positions_unit = unit.angstroms
positions0_dimensionless = np.array( protein_structure.positions / positions_unit )
positions1_dimensionless = np.array( molecule_structure.positions / positions_unit )

coordinates = np.vstack((positions0_dimensionless,positions1_dimensionless))
natoms = len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, positions_unit)

print(len(positions))
# Store in Structure object
pl_structure.positions = positions
# Save to PDB
pl_structure.save('pl_tmp.pdb',overwrite=True)

2649


In [73]:
# Solvate with PDBFixer
fixer = pdbfixer.PDBFixer('pl_tmp.pdb')
#fixer.findMissingResidues()
#fixer.findMissingAtoms()
#fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=unit.Quantity(10, unit.angstroms),
                 ionicStrength=unit.Quantity(50, unit.millimolar))

In [74]:
# Load PDBFixer object back to Structure
tmp_structure = parmed.openmm.load_topology(fixer.topology, xyz=fixer.positions)
print(tmp_structure.topology)

<Topology; 3 chains, 10084 residues, 32354 atoms, 22440 bonds>


In [75]:
#Store positions, topology, and box vectors for solvated system
full_positions = tmp_structure.positions
full_topology = tmp_structure.topology
full_box = tmp_structure.box

print(len(full_positions), full_topology, full_box)

32354 <Topology; 3 chains, 10084 residues, 32354 atoms, 22440 bonds> [ 69.92889222  69.92889222  69.92889222  90.          90.          90.        ]


In [76]:
# Remove from ligand from tmp Structure
tmp_structure.strip(':LIG')
tmp_structure.save('nomol_tmp.pdb',overwrite=True)

In [77]:
nomol = parmed.load_file('nomol_tmp.pdb')
print(nomol.topology, nomol.box, len(nomol.positions))

<Topology; 2 chains, 10083 residues, 32339 atoms, 22440 bonds> [ 69.929  69.929  69.929  90.     90.     90.   ] 32339


In [78]:
# Regenerate OpenMM System to parameterize solvent
nomol_system = forcefield.createSystem(nomol.topology, rigidWater=False)
# Regenerate parameterized protein structure
solv_structure = parmed.openmm.load_topology( nomol.topology, nomol_system, 
                                                xyz=nomol.positions, box=nomol.box )

In [79]:
# Remerge with ligand structure
full_structure = solv_structure + molecule_structure
full_structure.box = nomol.box
full_structure.save('full_structure.pdb', overwrite=True)
print(len(full_structure.positions), full_structure.topology, full_structure.box)
# Regenerate OpenMM system with ParmEd
system = full_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms, 
                                constraints=app.HBonds)

32354 <Topology; 3 chains, 10084 residues, 32354 atoms, 22455 bonds> [ 69.929  69.929  69.929  90.     90.     90.   ]


In [80]:
# Write out complex to oeb.gz
complex_mol = oechem.OEMol()
pkl_dict = pickle.dumps(full_structure.__getstate__())
encoded_dict = base64.b64encode(pkl_dict)
encoded_system = openmm.XmlSerializer.serialize(system).encode()
with oechem.oemolistream('full_structure.pdb') as ifs:
    if not oechem.OEReadMolecule(ifs, complex_mol):
        raise RuntimeError("Error reading complex pdb")
    with oechem.oemolostream('testcomplex.oeb.gz') as ofs:
        complex_mol.SetData(oechem.OEGetTag('system'), encoded_system)
        complex_mol.SetData(oechem.OEGetTag('structure'), encoded_dict)
        oechem.OEWriteConstMolecule(ofs, complex_mol)

In [81]:
print(complex_mol.GetData().keys())

dict_keys(['system', 'structure'])


In [82]:
#Read complex.oeb.gz back in
oebmol = oechem.OEMol()
with oechem.oemolistream('testcomplex.oeb.gz') as ifs:
    if not oechem.OEReadMolecule(ifs, oebmol):
        raise RuntimeError("Error reading complex") 

In [83]:
encoded_system = oebmol.GetData(oechem.OEGetTag('system'))
system = openmm.XmlSerializer.deserialize(encoded_system)
encoded_structure = oebmol.GetData(oechem.OEGetTag('structure'))           
decoded_structure = base64.b64decode(encoded_structure)
struct_dict = pickle.loads(decoded_structure)
structure = parmed.structure.Structure()
structure.__setstate__(struct_dict)
print(structure)

<Structure 32354 atoms; 10084 residues; 22455 bonds; PBC (orthogonal); parametrized>


In [84]:
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)
simulation = app.Simulation(structure.topology, system, integrator)
print(simulation.context.getPlatform().getName())

CPU


In [85]:
simulation.context.setPositions(structure.positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

In [86]:
simulation.minimizeEnergy(maxIterations=20)
st = simulation.context.getState(getPositions=True,getEnergy=True)
print('Minimized energy is {}'.format(st.getPotentialEnergy()))
with open('minimized.pdb', 'w') as minout:
    app.PDBFile.writeFile(simulation.topology, st.getPositions(), minout)

Minimized energy is -454994.06768531026 kJ/mol


In [87]:
min_state = simulation.context.getState(getPositions=True, getVelocities=True, getParameters=True,
                                    getForces=True, getParameterDerivatives=True, getEnergy=True,
                                    enforcePeriodicBox=True)

Xml should serialize State, System, Integrator, Forcefield objects

In [22]:
minpdb = open('minimized.pdb', 'rb')
encoded_min_state = openmm.XmlSerializer.serialize(min_state).encode()
encoded_min_system = openmm.XmlSerializer.serialize(system).encode()

In [23]:
# Write out complex to oeb.gz
min_mol = oechem.OEMol()
with oechem.oemolistream('minimized.pdb') as ifs:
    if not oechem.OEReadMolecule(ifs, min_mol):
        raise RuntimeError("Error reading complex pdb")
    with oechem.oemolostream('minimized.oeb.gz') as ofs:
        min_mol.SetData(oechem.OEGetTag('system'), encoded_min_system)
        min_mol.SetData(oechem.OEGetTag('state'), encoded_min_state)
        min_mol.SetData(oechem.OEGetTag('pdb') , minpdb.read())
        oechem.OEWriteConstMolecule(ofs, min_mol)

In [24]:
%ls -lht *minimized.*

-rw-r--r-- 1 nathanlim nathan-x220 4.4M Feb 19 20:44 minimized.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 20:44 minimized.pdb
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 17:25 minimized.tmp


In [25]:
min_mol1 = oechem.OEMol()
with oechem.oemolistream('minimized.oeb.gz') as ifs:
    if not oechem.OEReadMolecule(ifs, min_mol1):
        raise RuntimeError("Error reading complex pdb")
    print(min_mol1.GetData().keys())

dict_keys(['state', 'system', 'pdb'])


In [26]:
encoded_pdb = min_mol1.GetData(oechem.OEGetTag('pdb'))
f = io.StringIO(encoded_pdb)
pdbfile = app.PDBFile(f)
print(pdbfile.topology)
print(pdbfile.positions[0])

<Topology; 3 chains, 10084 residues, 32354 atoms, 22455 bonds>
(3.3494, 1.1226, 4.450200000000001) nm


In [27]:
min_pdb = app.PDBFile('minimized.pdb')
print(min_pdb.topology)
min_pdb.positions[0]

<Topology; 3 chains, 10084 residues, 32354 atoms, 22455 bonds>


Quantity(value=(3.3494, 1.1226, 4.450200000000001), unit=nanometer)

In [28]:
encoded_system = min_mol1.GetData(oechem.OEGetTag('system'))
min_system = openmm.XmlSerializer.deserialize(encoded_system)
serialized_state = min_mol1.GetData(oechem.OEGetTag('state'))
min_state1 = openmm.XmlSerializer.deserialize( serialized_state )

In [29]:
min_state1.getPeriodicBoxVectors() == min_state.getPeriodicBoxVectors()
min_state1.getForces() == min_state.getForces()
min_state1.getPositions() == min_state.getPositions()
min_state1.getParameters() == min_state.getParameters()
min_state1.getPositions() == min_state.getPositions()
min_state1.getPotentialEnergy() == min_state.getPotentialEnergy()
min_state1.getVelocities() == min_state.getVelocities()
min_state1.getTime() == min_state.getTime()

True

In [48]:
integrator1 = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)

In [49]:
simulation1 = app.Simulation(pdbfile.topology, system, integrator1)
print(simulation.context.getPlatform().getName())

CPU


In [50]:
simulation1.context.setState(min_state1)

In [51]:
traj_reporter = mdtraj.reporters.HDF5Reporter('simulation.h5', 100)
cdf_reporter = mdtraj.reporters.NetCDFReporter('simulation.nc', 100)
simulation1.reporters.append(traj_reporter)
simulation1.reporters.append(cdf_reporter)

In [52]:
simulation1.step(1000)

In [215]:
simulation1.reporters.

In [106]:
traj_reporter.close()
cdf_reporter.close()

In [54]:
state = simulation1.context.getState(getPositions=True, getVelocities=True, getParameters=True,
                                    getForces=True, getParameterDerivatives=True, getEnergy=True,
                                    enforcePeriodicBox=True)
with open('simulation.pdb', 'w') as simout:
    app.PDBFile.writeFile(simulation1.topology, state.getPositions(), simout)
state_pdb = app.PDBFile('simulation.pdb')
print(state_pdb.positions[0])

(3.439, 1.0102, 4.459300000000001) nm


In [89]:
%ls -lht

total 44M
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 21:03 minimized.pdb
-rw-r--r-- 1 nathanlim nathan-x220 4.3M Feb 19 21:02 testcomplex.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 21:02 full_structure.pdb
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 21:02 nomol_tmp.pdb
-rw-r--r-- 1 nathanlim nathan-x220 210K Feb 19 21:02 pl_tmp.pdb
-rw-r--r-- 1 nathanlim nathan-x220  36K Feb 19 21:02 mol_parameters.ffxml
-rw-r--r-- 1 nathanlim nathan-x220 5.6K Feb 19 21:02 smirff_mol.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220  27K Feb 19 21:01 openmmcube.ipynb
-rw-r--r-- 1 nathanlim nathan-x220 4.4M Feb 19 21:00 simulationNC.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 20:57 simulation.pdb
-rw-r--r-- 1 nathanlim nathan-x220 3.8M Feb 19 20:57 simulation.nc
-rw-r--r-- 1 nathanlim nathan-x220 5.5M Feb 19 20:57 simulation.h5
-rw-r--r-- 1 nathanlim nathan-x220 4.4M Feb 19 20:44 minimized.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 4.4M Feb 19 20:28 simulationh5.oeb.gz
-rw-r--r-- 1 na

In [158]:
with open('simulation.nc', 'rb') as infile:
    contents = str(infile.read())
    print(type(   contents   ))
    print(    len(   contents   )    )

<class 'str'>
9095747


In [193]:
h5traj = mdtraj.load('simulation.h5')
h5traj

<mdtraj.Trajectory with 10 frames, 32354 atoms, 10084 residues, and unitcells at 0x7f2a80590828>

In [200]:
pkl_traj = pickle.dumps(h5traj)
b64_traj = base64.b64encode(pkl_traj)

In [201]:
# Write out complex to oeb.gz
outmol = oechem.OEMol()
pdb = open('simulation.pdb','rb')
#traj = open('simulation.h5', 'rb') 
#print(type(traj.read()))
encoded_state = openmm.XmlSerializer.serialize(state).encode()
encoded_system = openmm.XmlSerializer.serialize(system).encode()
with oechem.oemolistream('simulation.pdb') as ifs:
    if not oechem.OEReadMolecule(ifs, outmol):
        raise RuntimeError("Error reading complex pdb")
    with oechem.oemolostream('simulationH5.oeb.gz') as ofs:
        outmol.SetData(oechem.OEGetTag('system'), encoded_min_system)
        outmol.SetData(oechem.OEGetTag('state'), encoded_min_state)
        outmol.SetData(oechem.OEGetTag('pdb') , pdb.read())
        outmol.SetData(oechem.OEGetTag('traj') , b64_traj)
        #outmol.SetData(oechem.OEGetTag('traj_nc') , str(traj.read()) )
        oechem.OEWriteConstMolecule(ofs, outmol)
#pdb.close()
traj.close()

In [202]:
print(outmol.GetData().keys())

dict_keys(['state', 'system', 'pdb', 'traj'])


In [203]:
%ls -lht simulation*

-rw-r--r-- 1 nathanlim nathan-x220 8.8M Feb 19 21:32 simulationH5.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 8.7M Feb 19 21:21 simulation_out.nc
-rw-r--r-- 1 nathanlim nathan-x220 8.6M Feb 19 21:20 simulationNC2.oeb.gz
-rw-r--r-- 1 nathanlim nathan-x220 2.5M Feb 19 20:57 simulation.pdb
-rw-r--r-- 1 nathanlim nathan-x220 3.8M Feb 19 20:57 simulation.nc
-rw-r--r-- 1 nathanlim nathan-x220 5.5M Feb 19 20:57 simulation.h5


In [207]:
simmol = oechem.OEMol()
with oechem.oemolistream('simulationH5.oeb.gz') as ifs:
    if not oechem.OEReadMolecule(ifs, simmol):
        raise RuntimeError("Error reading complex pdb")
        
    encoded_pdb = simmol.GetData(oechem.OEGetTag('pdb'))
    pdbfile = app.PDBFile(io.StringIO(encoded_pdb))
    print(pdbfile.topology)
    print(pdbfile.positions[0])
    
    encoded_traj = simmol.GetData(oechem.OEGetTag('traj'))
    print(type(traj))
    print(len(traj))
    decoded_traj = base64.b64decode(encoded_traj)
    regen_traj = pickle.loads(decoded_traj)
    regen_traj.save('regen_ttraj.h5')
    #with open('simulation_out.nc', 'wb') as outfile:
    #    outfile.write(traj.encode())

<Topology; 3 chains, 10084 residues, 32354 atoms, 22455 bonds>
(3.439, 1.0102, 4.459300000000001) nm
<class 'str'>
8859284


In [210]:
h5traj.n_atoms == regen_traj.n_atoms

True